<a href="https://colab.research.google.com/github/orca3/llm-model-serving/blob/main/ch02/ch2_Run_LLM_With_vLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import gc
import time

# Unload models and clean up gpu memory cache
def free_gpu(model):
  if model:
    # Removes the reference to the model's memory,
    # making it eligible for garbage collection.
    del model

  # Release any cached GPU memory that's no longer needed.
  if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()

  # Trigger garbage collection to ensure memory is fully released.
  gc.collect()

free_gpu(None)


In [2]:
!pip install vllm
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.4/326.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 110.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.

In [3]:
# prompt: give me code to Run Qwen2.5-0.5B model with vLLM framework, time the model generation.

import time
from vllm import LLM, SamplingParams

model_name = "Qwen/Qwen2.5-0.5B"  # Replace with the actual model name if different

# Load model with vLLM.
llm = LLM(model=model_name, dtype="float16")

# Define the prompt.
prompt = """You are an expert AI historian writing a detailed chapter for a book titled "The Evolution of Human-AI Collaboration."

Begin by summarizing the early stages of artificial intelligence in the 1950s, touching on symbolic logic and rule-based systems. Then transition into the rise of machine learning, particularly deep learning in the 2010s.

Afterward, describe how large language models like GPT transformed human-computer interaction, enabling applications in education, creative writing, customer support, and software development.

Finally, reflect on the societal and ethical implications of AI, such as misinformation, bias, and the alignment problem.

Write in a formal tone, with rich detail and examples in each era."""

# Create sampling parameters.
sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens=128)

# Time the model generation.
start_time = time.time()
outputs = llm.generate([prompt], sampling_params)
end_time = time.time()

# Print the results.
for output in outputs:
  print(f"Generated text: {output}")
  print(f"Time taken: {end_time - start_time:.2f} seconds")

free_gpu(llm)


INFO 05-04 08:32:38 [__init__.py:239] Automatically detected platform cuda.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/681 [00:00<?, ?B/s]

WARNING 05-04 08:32:49 [config.py:2972] Casting torch.bfloat16 to torch.float16.
INFO 05-04 08:33:07 [config.py:717] This model supports multiple tasks: {'reward', 'score', 'generate', 'embed', 'classify'}. Defaulting to 'generate'.
WARNING 05-04 08:33:07 [arg_utils.py:1658] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
INFO 05-04 08:33:07 [llm_engine.py:240] Initializing a V0 LLM engine (v0.8.5.post1) with config: model='Qwen/Qwen2.5-0.5B', speculative_config=None, tokenizer='Qwen/Qwen2.5-0.5B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='auto', reaso

tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

...

INFO 05-04 08:33:19 [loader.py:458] Loading weights took 1.37 seconds
INFO 05-04 08:33:20 [model_runner.py:1140] Model loading took 0.9277 GiB and 6.463576 seconds
INFO 05-04 08:33:22 [worker.py:287] Memory profiling takes 2.12 seconds
INFO 05-04 08:33:22 [worker.py:287] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.90) = 13.27GiB
INFO 05-04 08:33:22 [worker.py:287] model weights take 0.93GiB; non_torch_memory takes 0.05GiB; PyTorch activation peak memory takes 1.44GiB; the rest of the memory reserved for KV Cache is 10.85GiB.
INFO 05-04 08:33:22 [executor_base.py:112] # cuda blocks: 59265, # CPU blocks: 21845
INFO 05-04 08:33:22 [executor_base.py:117] Maximum concurrency for 32768 tokens per request: 28.94x
INFO 05-04 08:33:27 [model_runner.py:1450] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CL

Capturing CUDA graph shapes:   0%|          | 0/35 [00:00<?, ?it/s]

INFO 05-04 08:33:58 [model_runner.py:1592] Graph capturing finished in 31 secs, took 0.15 GiB
INFO 05-04 08:33:58 [llm_engine.py:437] init engine (profile, create kv cache, warmup model) took 38.58 seconds


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Generated text: RequestOutput(request_id=0, prompt='You are an expert AI historian writing a detailed chapter for a book titled "The Evolution of Human-AI Collaboration."\n\nBegin by summarizing the early stages of artificial intelligence in the 1950s, touching on symbolic logic and rule-based systems. Then transition into the rise of machine learning, particularly deep learning in the 2010s.\n\nAfterward, describe how large language models like GPT transformed human-computer interaction, enabling applications in education, creative writing, customer support, and software development.\n\nFinally, reflect on the societal and ethical implications of AI, such as misinformation, bias, and the alignment problem.\n\nWrite in a formal tone, with rich detail and examples in each era.', prompt_token_ids=[2610, 525, 458, 6203, 15235, 42968, 4378, 264, 11682, 12453, 369, 264, 2311, 24849, 330, 785, 37221, 315, 11097, 6691, 40, 86587, 2217, 11135, 553, 28285, 4849, 279, 4124, 17628, 315, 20443, 11

In [4]:
free_gpu(llm)

In [5]:
# --- Basic Model Serving (transformers) ---

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

start_time_basic = time.time()

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B", device_map="auto", trust_remote_code=True)

# Create the pipeline.
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

outputs_basic = generator(prompt, max_length=128, temperature=0.8, top_p=0.95)
end_time_basic = time.time()

print("\n---- Basic Model Serving Results ----")
for output in outputs_basic:
    print(f"Generated text: {output['generated_text']}")
    print(f"Time taken: {end_time_basic - start_time_basic:.2f} seconds")


print(f"\nLatency difference: {(end_time_basic - start_time_basic) - (end_time - start_time):.2f} seconds")

free_gpu(generator)


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is o


---- Basic Model Serving Results ----
Generated text: You are an expert AI historian writing a detailed chapter for a book titled "The Evolution of Human-AI Collaboration."

Begin by summarizing the early stages of artificial intelligence in the 1950s, touching on symbolic logic and rule-based systems. Then transition into the rise of machine learning, particularly deep learning in the 2010s.

Afterward, describe how large language models like GPT transformed human-computer interaction, enabling applications in education, creative writing, customer support, and software development.

Finally, reflect on the societal and ethical implications of AI, such as misinformation, bias, and the alignment problem.

Write in a formal tone, with rich detail and examples in each era. in a formal tone and provide specific examples to illustrate your points. The early stages of artificial intelligence in the 1950s were marked by the development of symbolic logic and rule-based systems. These systems 

NameError: name 'end_time_vllm' is not defined

In [6]:
print(f"\nLatency difference: {(end_time_basic - start_time_basic) - (end_time - start_time):.2f} seconds")



Latency difference: 18.46 seconds


In [7]:
free_gpu(llm)